<a href="https://colab.research.google.com/github/ftk1000/finapp/blob/master/yahoo_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# yahoo_fin
# 202103.03

In [3]:
from lxml import html  
import requests
# from exceptions import ValueError
from time import sleep
import json
import argparse
from collections import OrderedDict
# from time import sleep
import time
import pandas as pd
import urllib3
urllib3.disable_warnings()

In [11]:
def parse_etf(ticker):
    url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    url ="https://finance.yahoo.com/quote/%s?p=%s&.tsrc=fin-srch"%(ticker,ticker)
    response = requests.get(url, verify=False)
    print( "Parsing %s"%(url))
#     time.sleep(4)
    parser = html.fromstring(response.text)
    summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_json_response = requests.get(other_details_json_link)
    tt = summary_json_response.json()
    
    expRatio = get_ExpRatio(response)
    
    res = [ticker,
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
#     tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
    str(round(expRatio*100,2)) +"%"       ,
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt'] 
          ]
    return res

def get_ExpRatio(response):
    s = response.text
    s2 = "annualReportExpenseRatio"  #239750
    s2 = "feesExpensesInvestment"    #240971

    n1=s.find(s2)
    # print(n1)
    N=400
    n2=s[n1:(n1+N)].find("annualReportExpenseRatio")
    expr = float(s[n1+n2:(n1+n2+N)].split(":")[2].split(",")[0])
    return expr    

In [14]:
columns = ['ticker', 'fundFamily', 'category', 'yield', 'annualReportExpenseRatio',  'ytdReturn','beta3Year' ]
tickers = ['VOO', 'CLTL','SHV','VGSH','VGIT','GOVT','IEF','BND', 'JNK', 'ARKW', 'VGT','QQQ','VDE','EDV', 'BLV', 
           'VGLT', 'VMBS',  'VTIP',  'VCIT', 'VCSH', 'VTC', 'VTEB', 
           'VIG',  'VUG', 'VYM', 'BNDW']

tickers = ['vgt', 'arkg','arkw','vde','tan']
# ,'xom'  ,'spwr'
res = [parse_etf(ticker=tkr) for tkr in tickers ]
df=pd.DataFrame(res, columns=columns)
# df

Parsing https://finance.yahoo.com/quote/vgt?p=vgt&.tsrc=fin-srch
Parsing https://finance.yahoo.com/quote/arkg?p=arkg&.tsrc=fin-srch
Parsing https://finance.yahoo.com/quote/arkw?p=arkw&.tsrc=fin-srch
Parsing https://finance.yahoo.com/quote/vde?p=vde&.tsrc=fin-srch
Parsing https://finance.yahoo.com/quote/tan?p=tan&.tsrc=fin-srch


In [15]:
df

,ticker,fundFamily,category,yield,annualReportExpenseRatio,ytdReturn,beta3Year
0,vgt,Vanguard,Technology,0.83%,0.1%,4.12%,1.09
1,arkg,ARK ETF Trust,Health,0.77%,0.75%,4.84%,1.59
2,arkw,ARK ETF Trust,Technology,1.19%,0.79%,15.09%,1.48
3,vde,Vanguard,Equity Energy,4.53%,0.1%,31.92%,1.97
4,tan,Invesco,Miscellaneous Sector,0.09%,0.69%,3.85%,1.53


In [9]:
type(df['yield'][0])
# df.head(n=3)
df['cy'] = [float(df['yield'][k][:-1]) - float(df['annualReportExpenseRatio'][k][:-1]) for k in range(df.shape[0])]
df['ncy'] = [round(df['cy'][k]/float(df['beta3Year'][k]),2) if float(df['beta3Year'][k])>0 else -100 for k in range(df.shape[0])]

df = df.sort_values(by=['ncy'],ascending=False).reset_index()
df=df.drop(['index'], axis=1)
df

,ticker,fundFamily,category,yield,annualReportExpenseRatio,ytdReturn,beta3Year,cy,ncy
0,CLTL,Invesco,Ultrashort Bond,0.80%,0.08%,0.01%,0.01,0.72,72.00
1,SHV,iShares,Ultrashort Bond,0.60%,0.15%,0.00%,0.01,0.45,45.00
2,JNK,SPDR State Street Global Advisors,High Yield Bond,5.13%,0.4%,0.66%,0.64,4.73,7.39
3,VGSH,Vanguard,Short Government,1.13%,0.05%,-0.01%,0.20,1.08,5.40
4,VMBS,Vanguard,Intermediate Government,1.81%,0.05%,-0.45%,0.38,1.76,4.63
5,VTIP,Vanguard,Inflation-Protected Bond,1.19%,0.05%,0.78%,0.25,1.14,4.56
6,VCSH,Vanguard,Short-Term Bond,2.27%,0.05%,-0.21%,0.58,2.22,3.83
7,VYM,Vanguard,Large Value,3.19%,0.06%,6.19%,0.92,3.13,3.40
8,VDE,Vanguard,Equity Energy,4.53%,0.1%,31.92%,1.97,4.43,2.25
9,BND,Vanguard,Intermediate-Term Bond,2.23%,0.03%,-2.42%,1.03,2.20,2.14
